In [ ]:
import warnings
from itertools import product
from pathlib import Path

import geojson
import holidays
import numpy as np
import pandas as pd
import pyproj
from shapely import wkb
from shapely.geometry import shape
from shapely.ops import transform
from tqdm import tqdm

warnings.filterwarnings("ignore")
pd.options.plotting.backend = "plotly"

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
pd.set_option("display.width", 500)
pd.set_option("display.expand_frame_repr", False)

np.random.seed(2022)

In [ ]:
def reduce_mem_usage(df: pd.DataFrame) -> pd.DataFrame:
    """iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df.loc[:, col] = df.loc[:, col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df.loc[:, col] = df.loc[:, col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df.loc[:, col] = df.loc[:, col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df.loc[:, col] = df.loc[:, col].astype(np.int64)
            elif str(col_type)[:5] == "float":
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df.loc[:, col] = df.loc[:, col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df.loc[:, col] = df.loc[:, col].astype(np.float32)
                else:
                    df.loc[:, col] = df.loc[:, col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

# Предобработка данных

## Исходный датасет

- timestamp – временная метка публикации с точностью до часа
- lon, lat – координаты геопозиции с округлением до полигона 250х250 метров (географические долгота и широта, соответственно)
- likescount – количество отметок «лайк» у публикации
- commentscount – количество комментариев у публикации
- symbols_cnt – общее количество символов в публикации
- words_cnt – количество слов (осмысленных, не считая спецсимволов и прочую метаинформацию)
- hashtags_cnt – количество хештегов 
- mentions_cnt – количество упоминаний других пользователей
- links_cnt – количество ссылок
- emoji_cnt – количество эмодзи

In [ ]:
train_df = reduce_mem_usage(pd.read_csv("./../source/train_data_v2.csv"))

print(train_df.shape)
train_df.head()

## Преобразуем координаты

- переводим координаты в WKB формат;
- добавляем идентификаторы полигонов;

In [ ]:
train_df["polygon_center_wkb"] = train_df["point"].apply(bytes.fromhex)
train_df["polygon_id"] = train_df.groupby("polygon_center_wkb").ngroup()

train_df.head()

- создаем справчник полигонов.

In [ ]:
wgs84 = pyproj.CRS("EPSG:4326")
utm = pyproj.CRS("EPSG:3857")

wgs84_utm_project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
utm_wgs84_project = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform


def center_to_polygon(wgs84_point_wkb, d=125):
    wgs84_polygon = transform(
        utm_wgs84_project,
        transform(wgs84_utm_project, wkb.loads(wgs84_point_wkb)).buffer(distance=d, cap_style=3),
    )
    return wgs84_polygon.wkb


with open("./../source/spe_geometry.geojson") as f_src:
    spe_polygon = shape(geojson.load(f_src))

In [ ]:
polygons_df = train_df.groupby("polygon_center_wkb", as_index=False).first()[
    ["polygon_id", "lon", "lat", "polygon_center_wkb"]
]
polygons_df["polygon_wkb"] = polygons_df["polygon_center_wkb"].apply(center_to_polygon)
polygons_df["polygon_in_st_petersburg"] = (
    polygons_df["polygon_center_wkb"].apply(lambda point: spe_polygon.contains(wkb.loads(point))).astype(int)
)

polygons_df = polygons_df.astype(
    {
        "polygon_id": "int16",
        "polygon_center_wkb": "object",
        "polygon_wkb": "object",
        "polygon_in_st_petersburg": "int8",
    }
)

print(polygons_df.shape)
polygons_df.head()

In [ ]:
polygons_df.groupby("polygon_in_st_petersburg").agg(polygons_count=("polygon_id", "count"))

## Преобразуем временные метки

In [ ]:
def make_harmonic_features_cos(value, period=12):
    value *= 2 * np.pi / period
    return np.cos(value)


def make_harmonic_features_sin(value, period=12):
    value *= 2 * np.pi / period
    return np.sin(value)


russia_holidays = holidays.RU()

In [ ]:
calendar_df = pd.date_range("2019-01-01T00:00:00", "2020-02-29T23:00:00", freq="1H").to_frame(
    index=False, name="event_dttm"
)

calendar_df["event_ts"] = calendar_df["event_dttm"].astype(np.int64) // 10**9
calendar_df["event_time_id"] = calendar_df.index

calendar_df["year"] = calendar_df["event_dttm"].dt.year
calendar_df["month"] = calendar_df["event_dttm"].dt.month
calendar_df["day"] = calendar_df["event_dttm"].dt.day
calendar_df["weekday"] = calendar_df["event_dttm"].dt.weekday
calendar_df["hour"] = calendar_df["event_dttm"].dt.hour

calendar_df["weekend"] = calendar_df["event_dttm"].dt.weekday // 4
calendar_df["holidays"] = calendar_df["event_dttm"].apply(lambda x: int(x in russia_holidays))

calendar_df["month_cos"] = calendar_df["month"].apply(lambda x: make_harmonic_features_cos(x, 12))
calendar_df["month_sin"] = calendar_df["month"].apply(lambda x: make_harmonic_features_sin(x, 12))
calendar_df["weekday_cos"] = calendar_df["weekday"].apply(lambda x: make_harmonic_features_cos(x, 7))
calendar_df["weekday_sin"] = calendar_df["weekday"].apply(lambda x: make_harmonic_features_sin(x, 7))
calendar_df["hour_cos"] = calendar_df["hour"].apply(lambda x: make_harmonic_features_cos(x, 24))
calendar_df["hour_sin"] = calendar_df["hour"].apply(lambda x: make_harmonic_features_sin(x, 24))

calendar_df = reduce_mem_usage(calendar_df.set_index(["event_time_id", "event_ts", "event_dttm"]).reset_index())

print(calendar_df.shape)
calendar_df.head()

In [ ]:
print(f"event_dttm < '2020-02-01': {calendar_df[calendar_df['event_dttm'] < '2020-02-01'].shape[0]}")
print(f"event_dttm >= '2020-02-01': {calendar_df[calendar_df['event_dttm'] >= '2020-02-01'].shape[0]}")

In [ ]:
train_df = train_df.merge(calendar_df, how="left", left_on="timestamp", right_on="event_ts")
print(f"Null in 'event_time_id' column: {train_df['event_time_id'].isnull().sum()}")

print(train_df.shape)
train_df.head()

## Подготовка фичей

In [ ]:
posts_count_df = reduce_mem_usage(
    train_df.groupby(["polygon_id", "event_time_id"], as_index=False).agg(
        posts_count=("polygon_center_wkb", "count")
    )
)

print(posts_count_df.shape)
posts_count_df.head()

In [ ]:
posts_params_columns = [
    "likescount",
    "commentscount",
    "symbols_cnt",
    "words_cnt",
    "hashtags_cnt",
    "mentions_cnt",
    "links_cnt",
    "emoji_cnt",
]

# posts_params_aggregates = ["min", "max", "sum", "median", "mean", "std"]
posts_params_aggregates = ["mean", "std"]

In [ ]:
posts_params_df = reduce_mem_usage(
    train_df.groupby(["polygon_id", "event_time_id"], as_index=False).agg(
        {p: posts_params_aggregates for p in posts_params_columns}
    )
)
posts_params_df.columns = [c for c, _ in posts_params_df.columns if c not in posts_params_columns] + [
    f"{c}_{s}" for c, s in posts_params_df.columns if c in posts_params_columns
]

print(posts_params_df.shape)
posts_params_df.head()

## Формирование витрины

In [ ]:
mart_df = reduce_mem_usage(
    polygons_df.loc[polygons_df["polygon_in_st_petersburg"] == 1, ["polygon_id", "lon", "lat"]]
    .merge(
        calendar_df.drop(columns=["event_ts", "event_dttm"]),
        how="cross",
    )
    .merge(posts_count_df, on=["polygon_id", "event_time_id"], how="left")
    .merge(posts_params_df, on=["polygon_id", "event_time_id"], how="left")
    .fillna({"posts_count": 0})
    .astype({"posts_count": "int"})
    .set_index(["polygon_id", "event_time_id"])
    .reset_index()
)
mart_df.to_pickle("./_mart.pkl")

print(mart_df.shape)
mart_df.head()

In [ ]:
def get_series_lag_features(src_df: pd.DataFrame, col_names: list, lag_list: list) -> pd.DataFrame:
    df = src_df.copy()

    for col in tqdm(col_names):
        for lag in lag_list:
            df[f"{col}_lag_{lag // 24}"] = (
                df.sort_values(by=["polygon_id", "event_time_id"])
                .groupby(["polygon_id"])[col]
                .transform(lambda x: x.shift(lag))
            )

    return df


def get_series_roll_mean_features(
    src_df: pd.DataFrame, col_names: list, lag_list: list, roll_list: list
) -> pd.DataFrame:
    df = src_df.copy()

    for col in tqdm(col_names):
        for lag in lag_list:
            for roll in roll_list:
                df[f"{col}_lag_{lag // 24}_roll_{roll // 24}_mean"] = (
                    df.sort_values(by=["polygon_id", "event_time_id"])
                    .groupby(["polygon_id"])[f"{col}_lag_{lag // 24}"]
                    .transform(lambda x: x.rolling(window=roll).mean())
                )

    return df


def get_series_roll_std_features(
    src_df: pd.DataFrame, col_names: list, lag_list: list, roll_list: list
) -> pd.DataFrame:
    df = src_df.copy()

    for col in tqdm(col_names):
        for lag in lag_list:
            for roll in roll_list:
                df[f"{col}_lag_{lag // 24}_roll_{roll // 24}_std"] = (
                    df.sort_values(by=["polygon_id", "event_time_id"])
                    .groupby(["polygon_id"])[f"{col}_lag_{lag // 24}"]
                    .transform(lambda x: x.rolling(window=roll).std())
                )

    return df

In [ ]:
mart_df = pd.read_pickle("./_mart.pkl")

mart_df = reduce_mem_usage(
    get_series_lag_features(
        mart_df,
        ["posts_count"] + [f"{a}_{b}" for a, b in product(posts_params_columns, posts_params_aggregates)],
        # lag_list=[28 * 24, 35 * 24, 42 * 24],
        lag_list=[28 * 24],
    )
)
mart_df.to_pickle("./_mart_1.pkl")

print(mart_df.shape)
mart_df.head()

In [ ]:
mart_df = pd.read_pickle("./_mart_1.pkl")

mart_df = reduce_mem_usage(
    get_series_roll_mean_features(
        mart_df,
        ["posts_count"] + [f"{a}_{b}" for a, b in product(posts_params_columns, posts_params_aggregates)],
        # lag_list=[28 * 24, 35 * 24, 42 * 24],
        # roll_list=[8 * 24, 15 * 24, 22 * 24],
        lag_list=[28 * 24],
        roll_list=[15 * 24],
    )
)
mart_df.to_pickle("./_mart_2.pkl")

print(mart_df.shape)
mart_df.head()

In [ ]:
mart_df = pd.read_pickle("./_mart_2.pkl")

mart_df = reduce_mem_usage(
    get_series_roll_std_features(
        mart_df,
        ["posts_count"] + [f"{a}_{b}" for a, b in product(posts_params_columns, posts_params_aggregates)],
        # lag_list=[28 * 24, 35 * 24, 42 * 24],
        # roll_list=[8 * 24, 15 * 24, 22 * 24],
        lag_list=[28 * 24],
        roll_list=[15 * 24],
    )
)
mart_df.to_pickle("./_mart_3.pkl")

print(mart_df.shape)
mart_df.head()

In [ ]:
mart_df = pd.read_pickle("./_mart_3.pkl")

mart_df = reduce_mem_usage(
    mart_df[
        [
            col
            for col in mart_df.columns
            if col in polygons_df.columns
            or col in calendar_df.columns
            or col == "posts_count"
            or "_lag_" in col
            or "_roll_" in col
        ]
    ]
)

print(mart_df.shape)
mart_df.head()

## Подготовка валидационного и тестового датасетов

In [ ]:
valid_df = pd.read_csv("./../source/valid.csv")

valid_df["polygon_center_wkb"] = valid_df["point"].apply(bytes.fromhex)
valid_df["event_ts"] = valid_df["hour"]
valid_df["fact_posts_count"] = valid_df["sum"]

valid_df = valid_df.merge(
    polygons_df.loc[polygons_df["polygon_in_st_petersburg"] == 1, ["polygon_center_wkb", "polygon_id"]],
    how="inner",
    on="polygon_center_wkb",
).merge(calendar_df[["event_ts", "event_time_id"]], how="inner", on="event_ts")[
    ["polygon_id", "event_time_id", "fact_posts_count", "error"]
]

print(valid_df.shape)
valid_df.head()

In [ ]:
test_df = reduce_mem_usage(pd.read_csv("./../source/test.csv"))

test_df["polygon_center_wkb"] = test_df["point"].apply(bytes.fromhex)
test_df["event_ts"] = test_df["hour"]
test_df["fact_posts_count"] = test_df["sum"]

test_df = test_df.merge(
    polygons_df.loc[polygons_df["polygon_in_st_petersburg"] == 1, ["polygon_center_wkb", "polygon_id"]],
    how="inner",
    on="polygon_center_wkb",
).merge(calendar_df[["event_ts", "event_time_id"]], how="inner", on="event_ts")[
    ["polygon_id", "event_time_id", "fact_posts_count", "error"]
]

print(test_df.shape)
test_df.head()

## Подготовка обучающей выборки

In [ ]:
polygon_posts_df = mart_df.groupby("polygon_id", as_index=False)[["posts_count"]].sum()
polygon_posts_df["decile"] = pd.qcut(polygon_posts_df["posts_count"], q=10, labels=range(10))

polygon_sample_df = pd.concat(
    [polygon_posts_df[polygon_posts_df["decile"].eq(label)].sample(50) for label in range(10)]
)

mart_sample_df = mart_df.merge(polygon_sample_df[["polygon_id"]], how="inner", on="polygon_id")

print(mart_sample_df.shape)
mart_sample_df.head()

## Сохарнение результатов

In [ ]:
data_dir = Path("./../data")

if not data_dir.exists():
    data_dir.mkdir()

polygons_path = Path(data_dir, "polygons.pkl")
calendar_path = Path(data_dir, "calendar.pkl")

mart_path = Path(data_dir, "mart.pkl")
mart_sample_path = Path(data_dir, "mart_sample.pkl")

valid_path = Path(data_dir, "valid.pkl")
test_path = Path(data_dir, "test.pkl")

In [ ]:
polygons_df.to_pickle(polygons_path)
calendar_df.to_pickle(calendar_path)

mart_df.to_pickle(mart_path)
mart_sample_df.to_pickle(mart_sample_path)

valid_df.to_pickle(valid_path)
test_df.to_pickle(test_path)

## Визуализация части витрины

In [ ]:
pd.read_pickle(mart_sample_path).plot(x="event_time_id", y="posts_count", color="polygon_id")